# Import necessary packages

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F


# Set torch seed
torch.manual_seed(2023)

In [13]:
# Initialize Hyperparameters
HD_DIMENSION = 40000
NUM_CLASSES = 10
THRESHOLD = 0.0
BATCH_SIZE = 8
IMG_LEN = 28 * 28

In [3]:
# Load data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hd_device = torch.device('cpu')
# device = torch.device('mps')

print('Using device:', device)

# Load MNIST Data
transform = transforms.ToTensor()
train_data = torchvision.datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

test_data = torchvision.datasets.FashionMNIST(root='data', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

Using device: cuda


In [ ]:
# Function to generate HD vectors
def generate_hd_vectors(images, proj):

    # Adjust proj to sit between -1 and 1
    proj = (proj * 2) - 1

    # Send images to device
    images = images.to(hd_device)

    # Generate hypervectors and create binary vector
    hd_vectors = torch.sign(images @ proj.T)
    
    return hd_vectors.to(hd_device)

# Classify hd vectors using cosine similarity
def classify_hd_vectors(hd_vectors, hd_memory):
    distances = torch.cdist(hd_vectors, hd_memory, p=2)
    
    # Find the index of the minimum distance
    min_distances, min_indices = torch.min(distances, dim=1)
    return min_indices.to(hd_device)

def create_hd_memory(hd_vectors, labels, HD_DIMENSION):
    # Initialize HD Memory for classes
    hd_memory = torch.zeros((NUM_CLASSES, HD_DIMENSION), device=hd_device)

    # Create HD Memory
    for i, vec in enumerate(hd_vectors):
        hd_memory[train_data.targets[i]] += vec

    return torch.sign(hd_memory)


In [ ]:
# Create projection matrix
proj = torch.rand((HD_DIMENSION, IMG_LEN), device=hd_device, dtype=torch.float32)

# Create hyper vectors
hd_vectors = generate_hd_vectors(train_data.data.view(train_data.data.shape[0], -1) / 255, proj)

hd_memory = create_hd_memory(hd_vectors, train_data.targets, HD_DIMENSION)


In [ ]:
# Convert to binary for faster computation
def to_binary(hd_vectors, hd_memory):
    hd_vectors = (hd_vectors == 1).to(torch.float32)
    hd_memory = (hd_memory == 1).to(torch.float32)
    return hd_vectors, hd_memory

hd_vectors, hd_memory = to_binary(hd_vectors, hd_memory)

In [ ]:
# Testing training data
classifications = classify_hd_vectors(hd_vectors, hd_memory)

correct = torch.sum(classifications == train_data.targets.to(hd_device))
total = len(train_data.targets)
print(f"Accuracy on training data: {correct / total * 100}%")

In [ ]:
# Testing on test data
hd_vectors_test = generate_hd_vectors(test_data.data.view(test_data.data.shape[0], -1) / 255, proj)
hd_vectors_test = (hd_vectors_test == 1).to(torch.float32)

correct = 0
total = 0

classifications = classify_hd_vectors(hd_vectors_test, hd_memory)

for i, classification in enumerate(classifications):
    if classification == test_data.targets[i]:
        correct += 1
    total += 1

print(f"Accuracy on test data: {correct / total * 100}%")


In [ ]:
# Compare accuracy when applying random noise to images
noise_level = 0.85

noise_matrix = torch.randn((test_data.data.shape[0], IMG_LEN), device=hd_device) * noise_level

# Plot noise matrix
noisy_test_data = (test_data.data.view(test_data.data.shape[0], -1) / 255).to(hd_device) + noise_matrix

plt.imshow(noisy_test_data[0].view(28, 28).cpu(), cmap='gray')

hd_vectors_noise = generate_hd_vectors(noisy_test_data, proj)
hd_vectors_noise = (hd_vectors_noise == 1).to(torch.float32)
    

In [ ]:
# Test noisy test data
classifications = classify_hd_vectors(hd_vectors_noise, hd_memory)

correct = torch.sum(classifications == test_data.targets.to(hd_device))
total = len(test_data.targets)
print(f"Accuracy on test data with noise: {correct / total * 100}%")

# Defining and training an MLP for MNIST classification

In [ ]:
# Create simple neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(IMG_LEN, 512)
        self.fc2 = nn.Linear(512, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, NUM_CLASSES)
    
    def forward(self, x):
        x = x.view(-1, IMG_LEN)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        return x
    
# Create neural network
model = NeuralNetwork().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train neural network
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (BATCH_SIZE, 28*28)
        images = images.view(-1, 28 * 28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss and accuracy after each epoch
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in train_loader:
            images = images.view(-1, 28 * 28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy: {accuracy:.2f}%')


In [ ]:
# Testing loop to evaluate the model using test_loader
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    test_accuracy = 100 * correct / total
    print(f'Test Accuracy: {test_accuracy:.2f}%')

# Define and train a hybrid CNN feature extractor with hyperdimensional based feature extractor

In [20]:
from models import HDCNN

# Create HD-CNN
hd_cnn = HDCNN(HD_DIMENSION).to(device)

# Initialize class prototype dictionary for HD-CNN
class_prototypes = {}
for i in range(NUM_CLASSES):
    class_prototypes[i] = torch.zeros(HD_DIMENSION, device=device)


from torch.nn.functional import cosine_similarity

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, hd_cnn.parameters()), lr=0.001)

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        # Send data to device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        hd_vectors = hd_cnn(inputs)

        with torch.no_grad():
            # Incrementally update class prototypes
            for hd_vector, label in zip(hd_vectors, labels):
                # Binarize hd_vector
                hd_vector = torch.sign(hd_vector)

                # Update class prototype by summing the hd_vector
                class_prototypes[label.item()] += hd_vector

            # Binarize class prototypes again after updating
            for i in range(NUM_CLASSES):
                class_prototypes[i] = torch.sign(class_prototypes[i])

        # Compute cosine similarities with class prototypes for classification
        similarities = torch.stack([cosine_similarity(hd_vectors, class_prototypes[i]) for i in range(NUM_CLASSES)], dim=1)
        # print(similarities)
        # print(similarities.shape)
        # print(labels)
        # print(labels.shape)

        # Compute the loss
        loss = criterion(similarities, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update running loss and correct count
        running_loss += loss.item()
        _, predicted = torch.max(similarities.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate and print average loss and accuracy for this epoch
    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss}, Accuracy: {accuracy}%")

 

Epoch 1/10, Average Loss: 2.2179520716101435, Accuracy: 71.76333333333334%
Epoch 2/10, Average Loss: 2.217902425992287, Accuracy: 71.755%
Epoch 3/10, Average Loss: 2.2179659948510637, Accuracy: 71.69166666666666%
Epoch 4/10, Average Loss: 2.217938081692841, Accuracy: 71.715%
Epoch 5/10, Average Loss: 2.217919848733029, Accuracy: 71.79333333333334%
Epoch 6/10, Average Loss: 2.2179317110675876, Accuracy: 71.79333333333334%
Epoch 7/10, Average Loss: 2.2178828473818504, Accuracy: 71.74666666666667%
Epoch 8/10, Average Loss: 2.2179099361775285, Accuracy: 71.70833333333333%
Epoch 9/10, Average Loss: 2.2179823426877037, Accuracy: 71.835%
Epoch 10/10, Average Loss: 2.2179029917312882, Accuracy: 71.62%


## Test the model on the test set

In [21]:
# Testing loop to evaluate the model using test_loader
hd_cnn.eval()

correct = 0
total = 0
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)

    hd_vectors = hd_cnn(images)

    similarities = torch.stack([cosine_similarity(hd_vectors, class_prototypes[i]) for i in range(NUM_CLASSES)], dim=1)
    _, predicted = torch.max(similarities.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 64.24%


# Define and train a Vision Transformer for MNIST classification

In [ ]:
from vit_pytorch import ViT

v = ViT(
    image_size = 28,
    patch_size = 7,
    num_classes = 10,
    dim = 256,
    depth = 6,
    heads = 16,
    mlp_dim = 512,
    dropout = 0.5,
    emb_dropout = 0.1,
    channels = 1
).to(device)

# Train the model
import torch.optim as optim

optimizer = optim.Adam(v.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = v(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

    # Accuracy
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = v(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Accuracy: {accuracy:.2f}%')

# Test high dimensional vector MNIST classification with varying dimensions and noise levels

In [ ]:
# Test noisy test data with varying noise levels
noise_levels = np.arange(0, 1, 0.02)

# Make varying levels of noise from 10000 upto HD_DIMENSION
hd_dims = np.arange(10000, HD_DIMENSION, 10000)

total = len(test_data.targets)
hd_history = {}

# Initialize label-wise counters
label_counter = {i: 0 for i in range(10)}
label_total = {i: 0 for i in range(10)}

for hd_dim in hd_dims:
    # Create projection matrix
    proj = torch.rand((hd_dim, IMG_LEN), device=hd_device, dtype=torch.float32)

    # Create hyper vectors
    hd_vectors = generate_hd_vectors(train_data.data.view(train_data.data.shape[0], -1) / 255, proj)

    # Create HD Memory
    hd_memory = create_hd_memory(hd_vectors, train_data.targets, hd_dim)

    # Convert to binary for faster computation
    hd_vectors, hd_memory = to_binary(hd_vectors, hd_memory)

    accuracies_hd = []
    for noise_level in noise_levels:
        noise_matrix = torch.randn((test_data.data.shape[0], IMG_LEN), device=hd_device) * noise_level

        # Plot noise matrix
        noisy_test_data = (test_data.data.view(test_data.data.shape[0], -1) / 255).to(hd_device) + noise_matrix

        hd_vectors_noise = generate_hd_vectors(noisy_test_data, proj)
        hd_vectors_noise = (hd_vectors_noise == 1).to(torch.float32)

        classifications = classify_hd_vectors(hd_vectors_noise, hd_memory)

        correct = torch.sum(classifications.cpu() == test_data.targets)

        # Update label-wise counters
        for true, pred in zip(test_data.targets, classifications.cpu()):
            label_total[true.item()] += 1
            if true == pred:
                label_counter[true.item()] += 1
                
        accuracies_hd.append(correct / total * 100)

    hd_history[hd_dim] = accuracies_hd

# Calculate and print label-wise accuracies
label_accuracies = {label: (label_counter[label] / label_total[label]) * 100 for label in label_counter}
print("Label-wise accuracies:", label_accuracies)


# Test MLP with varying noise levels

In [ ]:
# Test neural network with progressively more noise
accuracies_net = []

for noise_level in noise_levels:

    # Test noisy test data
    correct = 0
    total = 0

    with torch.no_grad():
        for i, (images, labels) in enumerate(test_loader):
            images = images.view(-1, 28 * 28).to(device)
            labels = labels.to(device)

            # Add noise to images
            noise_matrix = torch.randn((images.shape[0], IMG_LEN), device=device) * noise_level
            images = images + noise_matrix

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracies_net.append(correct / total * 100)


# Test ViT with varying levels of noise

In [ ]:
acc_vit = []

for noise_level in noise_levels:
    
        # Test noisy test data
        correct = 0
        total = 0
    
        with torch.no_grad():
            for i, (images, labels) in enumerate(test_loader):
                images = images.to(device)
                labels = labels.to(device)
    
                # Add noise to images
                noise_matrix = torch.randn((images.shape[0], 1, 28, 28), device=device) * noise_level
                images = images + noise_matrix
    
                outputs = v(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        acc_vit.append(correct / total * 100)

# Display MNIST images with noise to get an idea of how noisy the images are to the human eye

In [ ]:
# Plot a random image from the test set with the various noise levels
plt.figure(figsize=(15, 15))

index = torch.randint(0, len(test_data.targets), size=(1,)).item()
for i, noise_level in enumerate(np.arange(0, 1, 0.1)):
    noise_matrix = torch.randn((test_data.data.shape[0], IMG_LEN)) * noise_level

    # Create noisy image
    noisy_test_data = (test_data.data.view(test_data.data.shape[0], -1) / 255) + noise_matrix

    # Create subplot and display the image
    plt.subplot(3, 4, i + 1)  # Assuming you have 10 images (0, 0.1, ..., 0.9), adjust the dimensions as necessary
    plt.imshow(noisy_test_data[index].view(28, 28), cmap='gray')
    plt.title(f'Noise Level: {noise_level:.1f}')
    plt.axis('off')  # Turn off axis numbers and ticks

plt.show()

In [ ]:
for hd_dim in hd_history:
    plt.plot(noise_levels, hd_history[hd_dim])
plt.plot(noise_levels, accuracies_net)
plt.plot(noise_levels, acc_vit)
legend = []
plt.legend(["HD Dimension: 10000", "HD Dimension: 20000", "HD Dimension: 30000", "HD Dimension: 40000", "HD Dimension: 50000", "HD Dimension: 60000", "HD Dimension: 70000", "Neural Network"])
plt.xlabel("Noise Level")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Noise Level for Neural Network")
plt.grid()